In [ ]:
"""
This notebook loads an example epi file from the storytelling dataset
(sub-hid000002_ses-pair02_task-storytelling3_run-03_bold_space-MNI152NLin2009cAsym_preproc.nii.gz)
and extracts affine and time series dimensions. It then loads the ICBM 2009c Nonlinear Asymmetric
whole brain mask (mni_icbm152_t1_tal_nlin_asym_09c_mask.nii) from (here:
https://www.bic.mni.mcgill.ca/ServicesAtlases/ICBM152NLin2009) and resamples it using the
affine and shape information from the example file.
"""

In [1]:
import numpy as np
from nilearn import image as nImage
import nibabel as nb

In [2]:
# example EPI image from fmriPrep output
epiFile = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/control_tasks/tmp/sub-hid000002_ses-pair02_task-storytelling3_run-03_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

# mask - downloaded from:
# https://www.bic.mni.mcgill.ca/ServicesAtlases/ICBM152NLin2009
maskFile = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/misc/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'

# path for saving out resampled mask
resampledMaskFile = '/dartfs-hpc/rc/lab/W/WheatleyT/f00589z/hyperscanning/misc/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask_RESAMPLED.nii'

In [3]:
# load example EPI image
tempImg = nImage.load_img(epiFile)

# set expected affine and shape based on example EPI image
expectedAffine = tempImg.affine
expectedShape = tempImg.shape[:3]
print('Expected affine: ' + str(expectedAffine))
print('Expected shape: ' + str(expectedShape))

# load mask
maskImg_OG = nImage.load_img(maskFile)
print('pre-resampling mask affine: ' + str(maskImg_OG.affine))
print('pre-resampling mask shape: ' + str(maskImg_OG.shape[:3]))

# re-sample mask
maskImg = nImage.resample_img(maskImg_OG,
                              target_affine=expectedAffine,
                              target_shape=expectedShape)
print('post-resampling mask affine: ' + str(maskImg.affine))
print('post-resampling mask shape: ' + str(maskImg.shape))

# Get data from resampled mask -- note that due to continuous interpolation,
# the mask will no longer be binary/boolean. Instead, each voxel will contain
# the mean of the 1s and 0s that were interpolated over in the resampling
# process. We take this as a proxy for the likelihood that a given voxel
# should be incorporated as a "brain" voxel (i.e., a 1) in the mask.
dataMask = maskImg.get_data()

# Set minimum 'likelihood' value for inclusion as a "brain" voxel in the mask
maskBoolCutoff = 0.1

# binarize
dataMask[dataMask >= maskBoolCutoff] = 1
dataMask[dataMask < maskBoolCutoff] = 0

# make a new image with the binarized mask data
maskImg = nb.Nifti1Image(dataMask,
                         maskImg.affine,
                         maskImg.header)

Expected affine: [[   3.    0.    0.  -96.]
 [   0.    3.    0. -132.]
 [   0.    0.    3.  -78.]
 [   0.    0.    0.    1.]]
Expected shape: (65, 77, 65)
pre-resampling mask affine: [[   1.    0.    0.  -96.]
 [   0.    1.    0. -132.]
 [   0.    0.    1.  -78.]
 [   0.    0.    0.    1.]]
pre-resampling mask shape: (193, 229, 193)
post-resampling mask affine: [[   3.    0.    0.  -96.]
 [   0.    3.    0. -132.]
 [   0.    0.    3.  -78.]
 [   0.    0.    0.    1.]]
post-resampling mask shape: (65, 77, 65)


<ipython-input-3-ff469948d2c1>:27: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  dataMask = maskImg.get_data()


In [4]:
maskImg.to_filename(resampledMaskFile)
